In [104]:
import geojson
from PIL import Image, ImageDraw, ImageFont
from pyproj import Proj, transform
import numpy as np
from simplification.cutil import simplify_coords, simplify_coords_vw, simplify_coords_vwp
import aggdraw
import copy
import timeit
import yaml
import pickle

DefaultStyle = {'color': 'red', 'width': 2}
sizeOfShortCoordinates = 0.001

proj_in = Proj(init='epsg:4326')
proj_out = Proj(init='epsg:3857')

In [105]:
def save_obj(obj, name):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)


def LoadData(fileName):
    with open(fileName, encoding='utf-8') as f:
        gj = geojson.load(f)
        
        for feature in gj['features']:
            
            minP = [999.0, 999.0]
            maxP = [0.0, 0.0]
            centerP = [0.0, 0.0]
            pointsNumber = 0.0
            
            
            for i in range(len(feature['geometry']['coordinates'])):
                a = feature['geometry']['coordinates'][i]
                
                for j in range(len(a)):
                    a[j] = ToConvertCoordinates(a[j])
                
                for b in a:
                    for c in b:
                        #Поиск минимальных и максимальных координат объекта
                        if c[0] < minP[0]:
                            minP[0] = c[0]
                        elif c[0] > maxP[0]:
                            maxP[0] = c[0]
                        if c[1] < minP[1]:
                            minP[1] = c[1]
                        elif c[1] > maxP[1]:
                            maxP[1] = c[1]
                        
                        #Поиск цента масс
                        centerP[0] += c[0]
                        centerP[1] += c[1]
                        pointsNumber += 1
                feature['geometry']['coordinates'][i] = a
            
            #Производит предварительное отсечение лишних вершин
            coordinates_short = []
            for a in feature['geometry']['coordinates']:
                a_short = []
                for b in a:
                    a_short.append(simplify_coords(b, sizeOfShortCoordinates))
                coordinates_short.append(a_short)
            feature['geometry']['coordinates_short'] = coordinates_short   
                    
            
            centerP[0] /= pointsNumber
            centerP[1] /= pointsNumber
            
            feature['properties']['minP'] = minP
            feature['properties']['maxP'] = maxP
            feature['properties']['centerP'] = centerP
            feature['properties']['w'] = maxP[0] - minP[0]
            feature['properties']['h'] = maxP[1] - minP[1]
            feature['properties']['density'] = pointsNumber / (maxP[0] - minP[0] + maxP[1] - minP[1]) / 2
        
        return gj['features']

def LoadStyles(fileName="styles.yaml", encoding='utf-8'):
    with open(fileName, 'r') as stream:
        try:
            styles = yaml.load(stream)
            if styles.get('rules')==None:
                prib=print('Файл стилей должен содержать rules')
                return None
            elif styles.get('exceptions')==None:
                prib=print('Файл стилей должен содержать exceptions')
                return None
            return styles
        except yaml.YAMLError as exc:
            print(exc)

def GetLevelStyle(styles, admin_level):
    levelStyle = copy.copy(DefaultStyle)
    
    if styles['rules'].get(admin_level) != None:
        levelStyle = styles['rules'][admin_level]
    
    if levelStyle.get('color')==None:
        levelStyle['color'] = DefaultStyle['color']
    if levelStyle.get('width')==None:
        levelStyle['width'] = DefaultStyle['width']
    return levelStyle

def GetCurrentStyle(styles, levelStyle, feature):
    currentStyle = copy.copy(levelStyle)
    
    exception = styles['exceptions'].get(feature['properties']['name'])
    
    if exception != None:
        if exception.get('color')!=None:
            currentStyle['color'] = exception['color']
        if exception.get('width')!=None:
            currentStyle['width'] = exception['width']
    return currentStyle

def ToConvertCoordinates(coordinatesArray):
    coordinatesArray = np.array(coordinatesArray)
    
    
    coordinatesArray[coordinatesArray < 0] += 360
    coordinatesArray -=30

    
    b=transform(proj_in, proj_out, coordinatesArray[:,0], coordinatesArray[:,1])
    coordinatesArray[:,0] = 90+b[0]/200000+15
    coordinatesArray[:,1] = 90-b[1]/200000
    
    return coordinatesArray



#Переводит координаты в формат для отрисовки
def ToNormalizeCoordinates(p, p0, size):
    x1 = (p[0]-p0[0]) * size
    y1 = (p[1]-p0[1]) * size
    return (x1, y1)

def ToCheckPointVisibility(p, p0, p1):
    if p[0] > p0[0] and p[1] > p0[1] and p[0] < p1[0] and p[1] < p1[1]:
        return True
    else:
        return False
def ToCheckPolygonVisibility(minP, maxP, p0, p1):
    if maxP[0] < p0[0] or maxP[1] < p0[1] or minP[0] > p1[0] or minP[1] > p1[1]:
        return False
    else:
        return True
    
def ToDrawPolygon(draw, coordinates, size, p0, p1, colour="red", width=1):
    previousP = -1
    norm_P0 = ToNormalizeCoordinates(copy.copy(p0), p0, size)
    norm_P1 = ToNormalizeCoordinates(copy.copy(p1), p0, size)
    points=[]
    
    for p in coordinates:
        points.append(ToNormalizeCoordinates(p, p0, size))
        continue
        #Не используемый код, предназначенный для рисования линиями
        if previousP == -1:
            previousP = ToNormalizeCoordinates(p, p0, size)
        else:
            currentP = ToNormalizeCoordinates(p, p0, size)
            
            if ToCheckPointVisibility(previousP, norm_P0, norm_P1) or ToCheckPointVisibility(currentP, p0, p1):
                draw.line((previousP[0], previousP[1], currentP[0], currentP[1]), fill=colour, width=int(width*size/100))
            previousP = currentP
        #
    draw.polygon(points,outline = colour)


def ToDrawPicture(x, y, z, P0, P1, pictureSize, activeLevelDictionary, styles):
    p0 = [0, 0]
    p1 = [0, 0]
    
    p0[0] = P0[0] + (P1[0] - P0[0]) * x / 2**z
    p0[1] = P0[1] + (P1[1] - P0[1]) * y / 2**z
    
    p1[0] = P0[0] + (P1[0] - P0[0]) * (x+1) / 2**z
    p1[1] = P0[1] + (P1[1] - P0[1]) * (y+1) / 2**z
    
    W = pictureSize
    H = pictureSize
    
    size = W / (p1[0] - p0[0])
    
    image = Image.new("RGBA", (int(W),int(H)), (250,250,255,255))
    draw = ImageDraw.Draw(image)
    
    for i in reversed(range(len(data))):
        dataLevel = data[i]
        if activeLevelDictionary.get(i)!=None and activeLevelDictionary[i]:
            currentStyle = GetLevelStyle(styles, i)
            
            for feature in dataLevel:
                
                if not ToCheckPolygonVisibility(feature['properties']['minP'], feature['properties']['maxP'], p0, p1):
                    continue
                
                coordinates = feature['geometry']['coordinates']
                if 1 / size / 10 > sizeOfShortCoordinates:
                    coordinates = feature['geometry']['coordinates_short']
                
                for a in coordinates:
                    for b in a:
                        b = simplify_coords(b, 1 / size / 10)
                        ToDrawPolygon(draw, b, size, p0, p1, currentStyle['color'], currentStyle['width'])
    del(draw)
    
    return image

In [106]:
data = []
#for i in range(10):
#    data.append([])
#data[2] = LoadData("admin_level_2.geojson")
#data[3] = LoadData("admin_level_3.geojson")
#data[6] = LoadData("admin_level_6.geojson")
#
#save_obj(data, 'data')

data = load_obj('data')
styles = LoadStyles()

In [107]:
import json
output_json = json.load(open('russian-cities.json', encoding='utf-8'))
output_json[0]['coords']

{'lat': '52.65', 'lon': '90.08333'}

In [108]:
#%timeit image = ToDrawPicture(1000, (100, 20), (180, 80), {2: True, 3: True, 6: False}, styles) #%timeit 
#%timeit image = ToDrawPicture(1000, (115, 55), (120, 60), {2: True, 3: True, 6: True}, styles)

image = ToDrawPicture(0, 0, 0, (90, 0), (200, 80), 256, {2: True, 3: True, 6: True}, styles)
#image.show()
#image.save("test.png", "PNG")

In [109]:
a=[float(output_json[0]['coords']['lat']), float(output_json[0]['coords']['lon'])]
a = ToConvertCoordinates([a])

In [117]:
def DravCitiesArray(draw, array, r, textSize):
    font = ImageFont.truetype("Roboto-Regular.ttf", textSize, encoding='UTF-8')

    for citie in array:
        x = citie['coords'][0][0]
        y = citie['coords'][0][1]
        draw.ellipse((x-r, y-r, x+r, y+r), fill=(128,128,128,128))
        draw.text((a[0][0],a[0][1]), citie['name'], font=font, fill=(128,128,128,128), encoding='UTF-8')



In [118]:
draw = ImageDraw.Draw(image)

CitiesArray = []
CitiesArray.append({'coords': a, 'name': output_json[0]['name']})

DravCitiesArray(draw, CitiesArray, 2, 14)

del(draw)
image.show()